In [ ]:
# --- 0. Numpy 버전 변경 ---
# 해당 쉘 실행 후 [런타임] - [세션 다시 시작]을 해야 함
print("Pinning NumPy to a stable version...")
!pip install numpy==1.26.4

print(f"\n✅ 0. Numpy 설치 완료 (세션을 다시 시작해주세요)")

In [ ]:
# --- 1. 환경 설정 ---

# 1. 원래 코드에서 사용한 라이브러리 설치
print("Installing necessary libraries...")
!pip install -q laion-clap torch torchaudio faiss-cpu pandas tqdm

# 2. 모든 라이브러리 임포트
import os
import torch
import numpy as np
import pandas as pd
import torchaudio
import faiss
from torch.utils.data import Dataset, DataLoader
from laion_clap import CLAP_Module
from tqdm.notebook import tqdm

# 3. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ 1. 환경 설정 완료")

In [ ]:
# --- 2. 경로 및 하이퍼파라미터 설정 ---

# === 입력 경로 (사용자 환경에 맞게 수정) ===
# Jamendo 데이터셋이 저장된 Google Drive의 기본 폴더 경로
DRIVE_JAMENDO_BASE_PATH = "/content/drive/MyDrive/Datasets/jamendo_autotag_mood_low"
# 메타데이터 파일 경로 (tsv 파일)
METADATA_PATH = "/content/drive/MyDrive/Datasets/jamendo_autotag_mood_low/csv/autotagging_moodtheme.tsv"

# === 출력 경로 (사용자 환경에 맞게 수정) ===
# 최종 결과물을 저장할 Google Drive 폴더 경로
OUTPUT_DIR = "/content/drive/MyDrive/Jamendo_Embeddings_Final"
OUTPUT_EMBEDDING_PATH = os.path.join(OUTPUT_DIR, "jamendo_clap_final.npy")
OUTPUT_METADATA_PATH = os.path.join(OUTPUT_DIR, "jamendo_clap_final.csv")
OUTPUT_FAISS_PATH = os.path.join(OUTPUT_DIR, "jamendo_clap_final.faiss")

# === 모델 및 처리 설정 ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
SAMPLING_RATE = 48000
DURATION_SEC = 30

# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ 2. 모든 경로 및 설정 완료. Device: {DEVICE}")

In [ ]:
# --- 3. CLAP 모델 로드 ---
print("Loading CLAP model...")
clap_model = CLAP_Module(enable_fusion=False)
clap_model.load_ckpt()
clap_model = clap_model.to(DEVICE)
clap_model.eval()

print("✅ 3. CLAP 모델 로드 완료")

In [ ]:
# --- 4. 데이터 준비 ---
import csv

# 1. 메타데이터 로드
print("Loading metadata and constructing file paths...")
try:
    # TSV 파일을 직접 읽어서 'path' 컬럼만 추출
    paths = []
    with open(METADATA_PATH, "r", encoding="utf-8", errors="ignore") as f:
        reader = csv.reader(f, delimiter="\t")
        header = next(reader)
        path_index = header.index("PATH") # 'PATH' 컬럼의 인덱스 찾기
        for row in reader:
            if len(row) > path_index:
                paths.append(row[path_index].strip())

    # DataFrame으로 변환
    metadata_df = pd.DataFrame({"relative_path": paths})
    print(f"Loaded {len(metadata_df)} paths from TSV.")

except Exception as e:
    print(f"❌ ERROR: Could not load metadata file. {e}")
    raise

# 2. 절대 경로 생성 및 파일 존재 여부 확인
def get_full_path(rel_path):
    # .mp3를 .low.mp3로 바꾸는 로직 포함
    low_path = rel_path.replace(".mp3", ".low.mp3") if rel_path.endswith(".mp3") else rel_path
    return os.path.join(DRIVE_JAMENDO_BASE_PATH, low_path)

metadata_df['full_path'] = metadata_df['relative_path'].apply(get_full_path)

print("\nVerifying file existence...")
file_exists = metadata_df['full_path'].apply(os.path.exists)
final_df = metadata_df[file_exists].copy()
file_paths = final_df['full_path'].tolist()
print(f"Found {len(final_df)} existing audio files to process.")


# 3. PyTorch Dataset 및 DataLoader 정의
class AudioPathDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx): return self.paths[idx]

def collate_fn_paths(batch):
    return [path for path in batch if path]

embedding_dataset = AudioPathDataset(file_paths)
embedding_dataloader = DataLoader(embedding_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_paths)

print("\n✅ 4. 데이터 준비 완료")

In [ ]:
# --- 5. 임베딩 생성 및 저장 ---

# 1. 가중치 적용 임베딩 함수
@torch.inference_mode()
def embed_audio_weighted(path, model, sr=48000):
    try:
        waveform, orig_sr = torchaudio.load(path)
        if orig_sr != sr:
            waveform = torchaudio.functional.resample(waveform, orig_sr, sr)

        # 30초보다 짧으면 패딩
        if waveform.shape[1] < sr * 30:
            waveform = torch.nn.functional.pad(waveform, (0, sr * 30 - waveform.shape[1]))
        else:
            waveform = waveform[:, :sr*30]

        first_10 = waveform[:, :sr*10]
        next_20  = waveform[:, sr*10:sr*30]

        emb_10 = model.get_audio_embedding_from_data(x=first_10.to(device), use_tensor=True)
        emb_20 = model.get_audio_embedding_from_data(x=next_20.to(device), use_tensor=True)
        weighted_emb = (2 * emb_10 + emb_20) / 3
        return weighted_emb.cpu().numpy()
    except Exception as e:
        print(f"❌ Failed to process {os.path.basename(path)}: {e}")
        return None

# 2. 임베딩 생성 루프 (체크포인트 기능 포함)
embeddings = []
valid_paths = []
CHECKPOINT_INTERVAL = 1000 # 1000개 파일마다 저장 (조절 가능)

print(f"--- Starting weighted embedding generation for {len(file_paths)} tracks ---")
progress_bar = tqdm(embedding_dataloader, desc="Generating Embeddings")

for i, path_batch in enumerate(progress_bar):
    for path in path_batch:
        emb = embed_audio_weighted(path, clap_model)
        if emb is not None:
            embeddings.append(emb)
            valid_paths.append(path)

    # 체크포인트 저장 로직
    if (i + 1) * BATCH_SIZE % CHECKPOINT_INTERVAL < BATCH_SIZE:
        print(f"\n💾 Saving checkpoint at {len(valid_paths)} files...")
        np.save(f"{OUTPUT_EMBEDDING_PATH}.part", np.vstack(embeddings))
        pd.DataFrame({'path': valid_paths}).to_csv(f"{OUTPUT_METADATA_PATH}.part", index=False)

# 3. 최종 결과 결합 및 저장
final_embeddings_np = np.vstack(embeddings)
output_meta_df = pd.DataFrame({'path': valid_paths})

print(f"\nEmbedding generation complete. Final shape: {final_embeddings_np.shape}")

# 4. 최종 파일 저장
print(f"\nSaving final files to: {OUTPUT_DIR}")
np.save(OUTPUT_EMBEDDING_PATH, final_embeddings_np)
output_meta_df.to_csv(OUTPUT_METADATA_PATH, index=False)

# 5. FAISS 인덱스 생성
dim = final_embeddings_np.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(final_embeddings_np.astype("float32"))
faiss.write_index(index, OUTPUT_FAISS_PATH)
print(f"FAISS index created at: {OUTPUT_FAISS_PATH}")

# 6. 임시 체크포인트 파일 삭제
if os.path.exists(f"{OUTPUT_EMBEDDING_PATH}.part"): os.remove(f"{OUTPUT_EMBEDDING_PATH}.part")
if os.path.exists(f"{OUTPUT_METADATA_PATH}.part"): os.remove(f"{OUTPUT_METADATA_PATH}.part")


print("\n✅ 5. 모든 임베딩 및 인덱스 파일 저장 완료")